# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import sqlite3

# import libraries for ML
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.wordnet import WordNetLemmatizer

# import libraries from sklearn
from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import classification_report

# import grid search
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
# connect to the database
conn = sqlite3.connect('DisasterResponse.db')
df = pd.read_sql('SELECT * FROM DisasterResponse', conn)
X = df['message']
y = df.drop(['message', 'original', 'genre'], axis=1)

In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [5]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [6]:
def tokenize(text):
    # normalize text
    text = text.lower()
    # remove punctuation
    text = re.sub(r'[^a-zA-A0-9]', ' ', text)
    # tokenize the text
    text = word_tokenize(text)
    # Remove stop words
    text = [w for w in text if w not in stop_words]
    
    # Lemmatization
    # Reduce words to their root form
    # lemmatize for nouns
    text = [lemmatizer.lemmatize(x) for x in text]
    # lemmatize for verbs
    text = [lemmatizer.lemmatize(x, pos='v') for x in text]
    
    return text

In [7]:
X[1]

'Is the Hurricane over or is it not over'

In [8]:
tokenize(X[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [9]:
X[3]

'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.'

In [10]:
tokenize(X[3])

['un',
 'report',
 'leogane',
 '80',
 '90',
 'destroy',
 'hospital',
 'st',
 'croix',
 'function',
 'need',
 'supply',
 'desperately']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('count_vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = pipeline
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [24]:
y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [25]:
y_test.iloc[:,0].value_counts()

1    4971
0    1529
2      54
Name: related, dtype: int64

In [26]:
np.unique(y_pred[:,0], return_counts=True)

(array([0, 1, 2], dtype=int64), array([ 886, 5614,   54], dtype=int64))

In [27]:
1573/(1537+962)

0.6294517807122849

In [28]:
print(classification_report(y_test.iloc[:,0], y_pred[:,0]))

              precision    recall  f1-score   support

           0       0.70      0.40      0.51      1529
           1       0.84      0.94      0.89      4971
           2       0.43      0.43      0.43        54

    accuracy                           0.81      6554
   macro avg       0.65      0.59      0.61      6554
weighted avg       0.80      0.81      0.79      6554



In [29]:
for col in range(y.shape[1]):
    print("Classification report for column " + str(col + 1) + " is:\n" + classification_report(y_test.iloc[:,col], y_pred[:,col]))

Classification report for column 1 is:
              precision    recall  f1-score   support

           0       0.70      0.40      0.51      1529
           1       0.84      0.94      0.89      4971
           2       0.43      0.43      0.43        54

    accuracy                           0.81      6554
   macro avg       0.65      0.59      0.61      6554
weighted avg       0.80      0.81      0.79      6554

Classification report for column 2 is:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5406
           1       0.88      0.50      0.64      1148

    accuracy                           0.90      6554
   macro avg       0.89      0.74      0.79      6554
weighted avg       0.90      0.90      0.89      6554

Classification report for column 3 is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accurac

C:\Users\Andre\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report for column 22 is:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6198
           1       0.68      0.11      0.19       356

    accuracy                           0.95      6554
   macro avg       0.81      0.55      0.58      6554
weighted avg       0.94      0.95      0.93      6554

Classification report for column 23 is:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6416
           1       0.67      0.04      0.08       138

    accuracy                           0.98      6554
   macro avg       0.82      0.52      0.54      6554
weighted avg       0.97      0.98      0.97      6554

Classification report for column 24 is:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6514
           1       0.00      0.00      0.00        40

    accuracy                           0.99      6554
   macro

### 6. Improve your model
Use grid search to find better parameters. 

In [22]:
pipeline.get_params()

{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('count_vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x00000155C4445708>)),
                                                   ('tfidf',
                                                    TfidfTransformer())]))])),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('count_vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000155C4445708>)),
                                                 ('tfidf',
                                                  TfidfTransformer())]))]),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'features__n_jobs': None,
 'features__

In [30]:
parameters = {
    'features__text_pipeline__count_vect__binary': [True, False],
    'features__text_pipeline__count_vect__max_df': [1],
    'features__text_pipeline__tfidf__norm': ['l2'],
    'features__text_pipeline__tfidf__smooth_idf': [True, False],
    'features__text_pipeline__tfidf__use_idf': [True],
    'clf__estimator__min_samples_split': [10]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = cv
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

KeyboardInterrupt: 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [65]:
# use gradient boosting classifier as opposed to random forset
# add ??? feature?

pipeline_2 = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('count_vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))
        ])),
        
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [66]:
model_2 = pipeline_2
model_2.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count_vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000155C4445708>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [67]:
y_pred_2 = model_2.predict(X_test)

In [68]:
for col in range(y.shape[1]):
    print("Classification report for column " + str(col + 1) + " is:\n" + classification_report(y_test.iloc[:,col], y_pred_2[:,col]))

Classification report for column 1 is:
              precision    recall  f1-score   support

           0       0.67      0.14      0.23      1529
           1       0.78      0.98      0.87      4971
           2       0.60      0.17      0.26        54

    accuracy                           0.78      6554
   macro avg       0.69      0.43      0.45      6554
weighted avg       0.75      0.78      0.71      6554

Classification report for column 2 is:
              precision    recall  f1-score   support

           0       0.90      0.97      0.94      5406
           1       0.78      0.52      0.62      1148

    accuracy                           0.89      6554
   macro avg       0.84      0.74      0.78      6554
weighted avg       0.88      0.89      0.88      6554

Classification report for column 3 is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accurac

In [70]:
pipeline_2.get_params()

{'memory': None,
 'steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('count_vect',
                                                    CountVectorizer(tokenizer=<function tokenize at 0x00000155C4445708>)),
                                                   ('tfidf',
                                                    TfidfTransformer())]))])),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('count_vect',
                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000155C4445708>)),
                                                 ('tfidf',
                                                  TfidfTransformer())]))]),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'features__n_jobs': None,
 'features__transfor

In [72]:
# tune parameters for pipline_2 above

parameters_2 = {
    'features__text_pipeline__count_vect__binary': [True, False],
    'features__text_pipeline__tfidf__smooth_idf': [True, False],
    'clf__estimator__learning_rate': [1, 0.7]
}

cv = GridSearchCV(pipeline_2, param_grid=parameters_2)

In [73]:
model_final = cv
model_final.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x00000155C4445708>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [1, 0.7],
                         'features__text_pipeline__count_vect__binary': [True,
                                                                         False],
         

In [75]:
y_pred_final = model_final.predict(X_test)

In [76]:
for col in range(y.shape[1]):
    print("Classification report for column " + str(col + 1) + " is:\n" + classification_report(y_test.iloc[:,col], y_pred_final[:,col]))

Classification report for column 1 is:
              precision    recall  f1-score   support

           0       0.69      0.13      0.22      1529
           1       0.78      0.98      0.87      4971
           2       0.67      0.15      0.24        54

    accuracy                           0.78      6554
   macro avg       0.71      0.42      0.44      6554
weighted avg       0.76      0.78      0.71      6554

Classification report for column 2 is:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5406
           1       0.81      0.48      0.60      1148

    accuracy                           0.89      6554
   macro avg       0.86      0.73      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Classification report for column 3 is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accurac

### 9. Export your model as a pickle file

source: 
https://stackabuse.com/scikit-learn-save-and-restore-models/

In [77]:
import pickle

In [81]:
import os
os.chdir('C:/Users/Andre/Desktop/Programming/Udacity/data_science/5/Disaster_response_project_2')

In [82]:
# Save to file in the current working directory
pkl_filename = "AdaBoostClassifier_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model_final, file)

In [86]:
# check that it works
# load model
with open(pkl_filename, 'rb') as file:
    my_pickle_model = pickle.load(file)

In [84]:
y_pred_final_loaded_model = my_pickle_model.predict(X_test)

In [85]:
y_pred_final_loaded_model[:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.